In [11]:
#import necessary libraries
import pandas as pd
import csv
import re
import validators
import emoji
import unidecode
import nltk
import pickle
nltk.download('stopwords')
import numpy as np
from sklearn.model_selection import train_test_split
from nltk.tokenize import WhitespaceTokenizer
from itertools import chain
import time
from sklearn.feature_extraction.text import TfidfVectorizer
import keras
from keras.preprocessing import sequence
import tensorflow as tf
from keras import Input
from keras import optimizers
from keras import backend as K
import torch
from keras import regularizers
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
from sklearn.datasets import make_blobs
from keras.models import Sequential
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from keras.layers import Dense, Activation, Dropout, Flatten
from keras.layers import Embedding, Conv1D, MaxPooling1D, GlobalMaxPooling1D 
from keras.utils import plot_model
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from matplotlib import pyplot
from keras.models import load_model
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_selection import SelectPercentile, f_classif

import tensorflow as tf
from tensorflow import keras
from keras import optimizers
from keras import regularizers
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten
from keras.layers import Embedding, Conv1D, MaxPooling1D, GlobalMaxPooling1D 
from keras.utils import np_utils
from keras.utils import plot_model
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from itertools import chain

from tqdm import tqdm
from gensim.models import fasttext
from gensim.test.utils import datapath
import os, re, csv, math, codecs, pickle, nltk
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

[nltk_data] Downloading package stopwords to C:\Users\Kirby
[nltk_data]     Wenceslao\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:
#Declare Stop Words

filipino_stopwords = set(
    """
akin
aking
ako
alin
am
amin
aming
ang
ano
anumang
apat
at
atin
ating
ay
bababa
bago
bakit
bawat
bilang
dahil
dalawa
dapat
din
dito
doon
gagawin
gayunman
ginagawa
ginawa
ginawang
gumawa
gusto
habang
hanggang
hindi
huwag
iba
ibaba
ibabaw
ibig
ikaw
ilagay
ilalim
ilan
inyong
isa
isang
itaas
ito
iyo
iyon
iyong
ka
kahit
kailangan
kailanman
kami
kanila
kanilang
kanino
kanya
kanyang
kapag
kapwa
karamihan
katiyakan
katulad
kaya
kaysa
ko
kong
kulang
kumuha
kung
laban
lahat
lamang
likod
lima
maaari
maaaring
maging
mahusay
makita
marami
marapat
masyado
may
mayroon
mga
minsan
mismo
mula
muli
na
nabanggit
naging
nagkaroon
nais
nakita
namin
napaka
narito
nasaan
ng
ngayon
ni
nila
nilang
nito
niya
niyang
noon
o
pa
paano
pababa
paggawa
pagitan
pagkakaroon
pagkatapos
palabas
pamamagitan
panahon
pangalawa
para
paraan
pareho
pataas
pero
pumunta
pumupunta
sa
saan
sabi
sabihin
sarili
sila
sino
siya
tatlo
tayo
tulad
tungkol
una
walang
""".split()
)

from nltk.corpus import stopwords
english_stopwords = stopwords.words('english')

search = "leni robredo bongbong marcos isko moreno domagoso manny pacman pacquiao ping lacson ernie abella leody de guzman norberto gonzales jose montemayor jr faisal mangondato"
candidatelist = search.split(" ")

In [13]:
#Load Models and files
    
with open('x_train.pkl', 'rb') as file:
    x_train = pickle.load(file)
    
with open('x_test.pkl', 'rb') as file:
    x_test = pickle.load(file)

with open('y_train.pkl', 'rb') as file:
    y_train = pickle.load(file)
    
with open('y_test.pkl', 'rb') as file:
    y_test = pickle.load(file)
    
with open('tweet_collection.pkl', 'rb') as file:
    tweet_collection = pickle.load(file)   
    
#Convert Labels from Strings to categorical Integers {Non-Hate = 1, Hate = 0}

df_ytrain = pd.DataFrame(y_train, columns = ['Label'])
df_ytest = pd.DataFrame(y_test, columns = ['Label'])

#flatten training set for fitting on tfidf
x_train_fit = list(chain.from_iterable(x_train))

mapping = {'Non-hate': 0, 'Hate': 1}
df_ytrain = df_ytrain.replace({'Label': mapping})
df_ytest = df_ytest.replace({'Label': mapping})


train_y = df_ytrain['Label'].tolist()
test_y = df_ytest['Label'].tolist()

np_train_y = np.asarray(train_y)
np_test_y = np.asarray(test_y)
    
list_len = [len(i) for i in x_train]
index_of_max = np.argmax(np.array(list_len))
max_sentence_len = list_len[index_of_max]

print(max_sentence_len)

#feed train set on vectorizer 
tokenizer = Tokenizer(num_words=100000, char_level=False)
tokenizer.fit_on_texts(x_train_fit)

#load model
thesix_it_model = load_model("fasttextcnn.h5")

36


In [14]:
print(len(y_test))

1536


In [51]:
def dummy_fun(doc):
    return doc

tfidf = TfidfVectorizer(
    analyzer='word',
    tokenizer=dummy_fun,
    preprocessor=dummy_fun,
    token_pattern=None)  

#classifier building/ fitting of training dataset to tfidf
fitted_training_x = tfidf.fit_transform(x_train)

#transform based on top 40 percent features
selector = SelectPercentile(f_classif, percentile = 40)
selector.fit(fitted_training_x, train_y)

#load model
cabasag_model = load_model("cabasag_model.h5")

In [52]:
#pre-process tweet input
def pre_process_tweet(tweet_input):
    #Step 1 - Extract Tweet from input
    #Tweet = tweet_input
    tweet = tweet_input.strip().replace("\n"," ")
    print("\nExtracted tweet: \n",tweet, sep = "")
    #Step 2 - Data Deidentification
    output = ""
    sentence = tweet.split(" ")
    for part in sentence:
        if not re.match(r"(^|[^@\w])@(\w{1,15})\b", part):
            if len(output) == 0:
                output = f"{part}"
            else:
                output = f"{output} {part}"

    tweets_de_identified = output
    print("\nDe-identified tweet: \n",tweets_de_identified, sep = "")
    #Step 3 - URL Removal
    output = ""
    sentence = tweets_de_identified.split(" ")
    for part in sentence:
        valid = validators.url(part)

        if (not valid == True):
            if len(output) == 0:
                output = f"{part}"
            else:
                output = f"{output} {part}"
                
    tweets_url_removed = output
    print("\nURL Removed tweet: \n",tweets_url_removed, sep = "")
    #Step 4 - Special Character Processing
    
    emoji_removed = emoji.replace_emoji(tweets_url_removed, replace="[emoji]")
    output = ""
    sentence = emoji_removed.split(" ")

    for part in sentence:
        if not (re.match(r"^[_\W]+$", part) or "[emoji]" in part):
            if len(output) == 0:
                output = f"{part}"
            else:
                output = f"{output} {part}"
    
    tweets_specialcharacters_removed = output
    print("\nSpecial Characters removed tweet: \n",tweets_specialcharacters_removed, sep = "")
    #Step 5 - Normalization, lowercase>removediacritics>remove numerics and symbols>stopwords
    
    #lowercase the text
    lowercased_input = tweets_specialcharacters_removed.lower()

    #remove diacritics
    diacritics_removed = unidecode.unidecode(lowercased_input)

    output = ""
    sentence = diacritics_removed.split(" ")

    for part in sentence:
        part = re.sub("[^A-Za-z ]+$", "", part)
        part = re.sub("^[^A-Za-z #]+", "", part)
        if not (len(part) <= 1 or re.match(r"[^#a-zA-Z]", part) or part in english_stopwords or part in filipino_stopwords or any(part in x for x in candidatelist)):     
            if len(output) == 0:
                output = f"{part}"
            else:
                output = f"{output} {part}"  
                
    tweets_normalized = output
    print("\nNormalized tweet: \n",tweets_normalized, sep = "")
    
    #Step 6 - Hashtag Processing, removing the hashtags from the tweet
    output = ""
    sentence = tweets_normalized.split(" ")

    for part in sentence:
        if not re.match(r"#(\w+)", part):
            if len(output) == 0:
                output = f"{part}"
            else:
                output = f"{output} {part}"
                
    tweets_hashtags_removed = output  
    print("\nHashtags removed tweet: \n",tweets_hashtags_removed, sep = "")
    #Step 7 - Tokenization
    tokenizer = WhitespaceTokenizer()
    
    output = tokenizer.tokenize(tweets_hashtags_removed)
    
    tweets_tokenized = output
    print("\nTokenized tweet: \n",tweets_tokenized, sep = "")
    print("\n",sep = "")
    return tweets_tokenized

In [53]:
def fasttextcnn_vectorizer(tokenized_tweet):
    tweet = " ".join(tokenized_tweet)
    transformed = tokenizer.texts_to_sequences([tweet])
    padded = keras.preprocessing.sequence.pad_sequences(transformed, maxlen=20)
    return padded

def fasttextcnn_predict(tweet, label):
    start_time = time.time()
    print("Input tweet: \n", tweet, sep = "")
    tokenized_tweet = pre_process_tweet(tweet)
    vectorized_tweet = fasttextcnn_vectorizer(tokenized_tweet)   
    
    prediction_noround = thesix_it_model.predict(vectorized_tweet)
    run_time = time.time() - start_time
    prediction = (prediction_noround > 0.5).astype(int)
    
    #model.predict
    classification = "Non-hate"

    if(prediction[0] == 1):
        classification = "Hate"
        
    print("\nUsing Binary Classification fastTextCNN Model\n~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~\nInput text:\n"
          , tweet, "\n\nTrue Label:\n- ",label,"\n\nModel Classification:\n- ", classification,
          "\n\nClassification Output:\n- ",prediction_noround,"\n- ", prediction[0], "\n\nRuntime: \n- ",
          run_time, " seconds.", sep = "")

In [54]:
def tfidf_vectorizer(tokenized_tweet):
    transformed = tfidf.transform([tokenized_tweet])
    return selector.transform(transformed).toarray()

def cabasag_predict(tweet, label):
    start_time = time.time()
    print("Input tweet: \n", tweet, sep = "")
    tokenized_tweet = pre_process_tweet(tweet)
    vectorized_tweet = tfidf_vectorizer(tokenized_tweet)
    
    prediction_noround = cabasag_model.predict(vectorized_tweet)
    run_time = time.time() - start_time
    prediction = np.argmax(prediction_noround)
    #model.predict
    
    classification = "Non-hate"

    if(prediction == 1):
        classification = "Hate"
        
    print("\nUsing Binary Classification fastTextCNN Model\n~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~\nInput text:\n"
          , tweet, "\n\nTrue Label:\n- ",label,"\n\nModel Classification:\n- ", classification,
          "\n\nClassification Output:\n- ",prediction_noround,"\n- ", prediction, "\n\nRuntime: \n- ",
          run_time, " seconds.", sep = "")

In [55]:
input_tweet = '''
@edupunay @PhilippineStar Hahaha ang nenega talaga ng kakampinks! Hindi nalang tanggapin na panalo na si Bongbong Marcos number 7 sa balota!! Hindi kami nag papabrainwash kay Leni Lugaw. 😛 Solid BBM kami! Solid UNITEAM ❤️💚
#BBMIsMyPresident2022
#BBMSARA2022
'''
input_label = "Hate"

fasttextcnn_predict(input_tweet, input_label)

Input tweet: 

@edupunay @PhilippineStar Hahaha ang nenega talaga ng kakampinks! Hindi nalang tanggapin na panalo na si Bongbong Marcos number 7 sa balota!! Hindi kami nag papabrainwash kay Leni Lugaw. 😛 Solid BBM kami! Solid UNITEAM ❤️💚
#BBMIsMyPresident2022
#BBMSARA2022


Extracted tweet: 
@edupunay @PhilippineStar Hahaha ang nenega talaga ng kakampinks! Hindi nalang tanggapin na panalo na si Bongbong Marcos number 7 sa balota!! Hindi kami nag papabrainwash kay Leni Lugaw. 😛 Solid BBM kami! Solid UNITEAM ❤️💚 #BBMIsMyPresident2022 #BBMSARA2022

De-identified tweet: 
Hahaha ang nenega talaga ng kakampinks! Hindi nalang tanggapin na panalo na si Bongbong Marcos number 7 sa balota!! Hindi kami nag papabrainwash kay Leni Lugaw. 😛 Solid BBM kami! Solid UNITEAM ❤️💚 #BBMIsMyPresident2022 #BBMSARA2022

URL Removed tweet: 
Hahaha ang nenega talaga ng kakampinks! Hindi nalang tanggapin na panalo na si Bongbong Marcos number 7 sa balota!! Hindi kami nag papabrainwash kay Leni Lugaw. 😛 Solid BBM 

ValueError: in user code:

    File "C:\Users\Kirby Wenceslao\anaconda3\lib\site-packages\keras\engine\training.py", line 1845, in predict_function  *
        return step_function(self, iterator)
    File "C:\Users\Kirby Wenceslao\anaconda3\lib\site-packages\keras\engine\training.py", line 1834, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\Kirby Wenceslao\anaconda3\lib\site-packages\keras\engine\training.py", line 1823, in run_step  **
        outputs = model.predict_step(data)
    File "C:\Users\Kirby Wenceslao\anaconda3\lib\site-packages\keras\engine\training.py", line 1791, in predict_step
        return self(x, training=False)
    File "C:\Users\Kirby Wenceslao\anaconda3\lib\site-packages\keras\utils\traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\Kirby Wenceslao\anaconda3\lib\site-packages\keras\engine\input_spec.py", line 264, in assert_input_

In [46]:
input_tweet = '''
@edupunay @PhilippineStar Hahaha ang nenega talaga ng kakampinks! Hindi nalang tanggapin na panalo na si Bongbong Marcos number 7 sa balota!! Hindi kami nag papabrainwash kay Leni Lugaw. 😛 Solid BBM kami! Solid UNITEAM ❤️💚
#BBMIsMyPresident2022
#BBMSARA2022
'''
input_label = "Hate"

cabasag_predict(input_tweet, input_label)

Input tweet: 

fuck you tangina mo 


Extracted tweet: 
fuck you tangina mo

De-identified tweet: 
fuck you tangina mo

URL Removed tweet: 
fuck you tangina mo

Special Characters removed tweet: 
fuck you tangina mo

Normalized tweet: 
fuck tangina

Hashtags removed tweet: 
fuck tangina

Tokenized tweet: 
['fuck', 'tangina']


1/1 [==============================] - 0s 18ms/step

Using Binary Classification fastTextCNN Model
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Input text:

fuck you tangina mo 


True Label:
- Hate

Model Classification:
- Hate

Classification Output:
- [[6.1006405e-05 9.9993896e-01]]
- 1

Runtime: 
- 0.050863027572631836 seconds.


In [10]:
#test runtime for all test data

def Average(runtime_list):
    sum_of_list = 0
    for i in range(len(runtime_list)):
        sum_of_list += runtime_list[i]
        
    average = sum_of_list/len(runtime_list)
    
    return average

In [11]:
#list how much 
fasttext_runtime = []
cabasag_runtime = []

for i in x_test:
    
    #fasttext
    
    start_time_f = time.time()
    
    vectorized_tweet_f = fasttextcnn_vectorizer(i) 
    prediction_f = thesix_it_model.predict(vectorized_tweet_f)
    
    run_time_f = time.time() - start_time_f
    
    #cabasag
    
    start_time_c = time.time()
    
    vectorized_tweet_c = tfidf_vectorizer(i)
    prediction_c = cabasag_model.predict(vectorized_tweet_c)
    
    run_time_c = time.time() - start_time_c
    
    fasttext_runtime.append(run_time_f)
    cabasag_runtime.append(run_time_c)

1/1 [==============================] - 0s 15ms/step


1/1 [==============================] - 0s 17ms/step


1/1 [==============================] - 0s 16ms/step


1/1 [==============================] - 0s 20ms/step


1/1 [==============================] - 0s 19ms/step


1/1 [==============================] - 0s 16ms/step


1/1 [==============================] - 0s 16ms/step


1/1 [==============================] - 0s 19ms/step


1/1 [==============================] - 0s 17ms/step


1/1 [==============================] - 0s 15ms/step
Average runtime for Binary 7030 fastTextCNN  0.04287239039937655
Average runtime for Binary 7030 TFIDF FFNN  0.04287239039937655


In [12]:
print("Average runtime for Binary 8020 fastTextCNN ",Average(fasttext_runtime))
print("Average runtime for Binary 8020 TFIDF FFNN ",Average(cabasag_runtime))

Average runtime for Binary 8020 fastTextCNN  0.04287239039937655
Average runtime for Binary 8020 TFIDF FFNN  0.04541309364140034


In [15]:
runtimes = pd.DataFrame(list(zip(fasttext_runtime,cabasag_runtime)), columns = ['fastText CNN', 'TFIDF FFNN'])
runtimes

,fastText CNN,TFIDF FFNN
0,0.040891,0.038896
1,0.039894,0.041888
2,0.038896,0.043882
3,0.041889,0.049867
4,0.044878,0.048871
...,...,...
1531,0.038896,0.044881
1532,0.036901,0.038896
1533,0.037898,0.038896
1534,0.036900,0.040892


In [16]:
runtimes.to_csv('7030runtime.csv', index=False)